In [ ]:
#Importing the required modules
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KernelDensity
from scipy.integrate import dblquad
from galpy.util.bovy_coords import *
from search_local import *
import time
import sys
sys.path.append('..')

#Setting up the quasi-isothermal instance for galpy
from galpy.df import quasiisothermaldf
from galpy.potential import MWPotential2014
from galpy.actionAngle import actionAngleStaeckel
from galpy.actionAngle import actionAngleAdiabatic

aAS= actionAngleStaeckel(pot=MWPotential2014,delta=0.45,c=True)
qdfS= quasiisothermaldf(1./3.,0.2,0.1,1.,1.,pot=MWPotential2014,aA=aAS,cutcounter=True)

In [ ]:
#Importing qdf load data
gaia_data = search_phase_space(0., 0., 0., 0., 0., 0., 0.5, 0.)

print('#stars', len(gaia_data))
print('min/max/mean (x)', min(gaia_data[:,0]), max(gaia_data[:,0]), np.mean(gaia_data[:,0]))
print('min/max/mean (y)', min(gaia_data[:,1]), max(gaia_data[:,1]), np.mean(gaia_data[:,1]))
print('min/max/mean (z)', min(gaia_data[:,2]), max(gaia_data[:,2]), np.mean(gaia_data[:,2]))
print('min/max/mean (vx)', min(gaia_data[:,3]), max(gaia_data[:,3]), np.mean(gaia_data[:,3]))
print('min/max/mean (vy)', min(gaia_data[:,4]), max(gaia_data[:,4]), np.mean(gaia_data[:,4]))
print('min/max/mean (vz)', min(gaia_data[:,5]), max(gaia_data[:,5]), np.mean(gaia_data[:,5]))

In [ ]:
#Defining the KDE function rather than importing to allow for quick and easy changes
def generate_KDE(inputs, ker, bw_multiplier=10):
    """
    NAME:
        generate_KDE
    
    PURPOSE:
        Given an NxM matrix for inputs and one of six avaliable ker strings, 
        outputs a function `input_DKE` that treats the density estimate as a 
        black box function that can be sampled.
    
    INPUT:
        inputs (ndarray) = An NxM matrix where N is the number of data 
                           points and M is the number of parameters.
        ker (string) = One of the 6 avaliable kernel types (gaussian, 
                       tophat, epanechnikov, exponential, linear, cosine).
        selection = a selection function that takes parallax to Sun and returns
                    fraction of stars that are left after selection;
                    takes array; takes parallax in physical units
    
    OUTPUT:
        input_KDE (function) = A blackbox function for the density estimate
                               used for sampling data.
                               
    HISTORY:
        2018-07-15 - Updated - Ayush Pandhi
    """
    #Scaling velocities with z-score
    inputs_std = np.nanstd(inputs, axis=0)
    i1, i2, i3, i4, i5, i6 = np.mean(inputs, axis=0)
    inputs_mean = np.hstack((i1, i2, i3, i4, i5, i6))
    inputs = (inputs - inputs_mean)/inputs_std
    
    #Optimizing bandwidth in terms of Scott's Multivariate Rule of Thumb
    N = inputs.shape[0]
    bw = bw_multiplier * np.nanstd(inputs) * N ** (-1/10.)
    
    #Fit data points to selected kernel and bandwidth
    kde = KernelDensity(kernel=ker, bandwidth=bw).fit(inputs)  
    
    def input_KDE(samples):
        """
        NAME:
            input_KDE
    
        PURPOSE:
            Given a QxM matrix for samples, evaluates the blackbox density
            estimate function at those points to output a 1xQ array of 
            density values.
    
        INPUT:
            samples (ndarray) = A QxM matrix where Q is the number of points 
                                at which the kde is being evaluated and M is 
                                the number of parameters.
                                
        OUTPUT:
            dens (ndarray) = A 1xQ array of density values for Q data points.
                               
        HISTORY:
            2018-07-15 - Updated - Ayush Pandhi
        """      
        #Converting cylindrical samples to cartesian
        x, y, z = cyl_to_rect(samples[:,0], samples[:,1], samples[:,2])
        vx, vy, vz = cyl_to_rect_vec(samples[:,3], samples[:,4], samples[:,5], samples[:, 1])
        samples = np.stack((x, y, z, vx, vy, vz), axis=1)
        
        #Scaling samples with standard deviation
        samples = (samples - inputs_mean)/inputs_std
        
        #Get the log density for selected samples and apply exponential to get normal probabilities
        log_dens = kde.score_samples(samples)
        dens = np.exp(log_dens)
        
        #Return a 1xQ array of normal probabilities for the selected sample
        return dens
    
    #Return a black box function for sampling
    return input_KDE

In [ ]:
#Generate kde for 6D qdf inputs
kde_gaia_epanechnikov = generate_KDE(gaia_data, 'epanechnikov')

In [ ]:
#input values for velocities (vR, vT, vz)
v_input = np.linspace(-400, 200, 100)

# -----------------
# Integrate over vR
# -----------------

def kde_gaia_cyl_vR(vT, vz, R, phi, z, vR):
    evaluation = kde_gaia_epanechnikov(np.array([[R, phi, z, vR, vT, vz]]))
    return evaluation

def integrate_over_vR(kde_gaia_epanechnikov, vR): #bounds of vT = [-300, 0], bounds of vz = [-100, 100]
    print ("Evaluating at vR =", vR)
    return dblquad(kde_gaia_cyl_vR, -300, 0, -100, 100, args=([8.3, 3.14, 0., vR]), epsabs=0.001)  # args=(R, phi, z, vR)

vR_output = np.ones([100])

print("With R, phi, z = 8.3, 3.14, 0., integrating over vT from [=300, 0] and vz from [-100, 100].")
print("KDE Evaluations along vR from [-400, 200] over 100 subintervals.")
print()

counter = 0
start_overall = time.time()

for v in np.nditer(v_input):
    counter += 1
    print("Evaluation:", counter)
    start = time.time()
    vR_output[counter - 1], error = integrate_over_vR(kde_gaia_epanechnikov, v)
    print("Value:", vR_output[counter - 1])
    end = time.time()
    print("Time to integrate: "'{}'"s".format(round(end - start, 2)))
    print("Time elapsed: "'{}'" min".format(round((end - start_overall)/60, 2)))
    print()
    
# -----------------
# Integrate over vT
# -----------------

def kde_gaia_cyl_vT(vR, vz, R, phi, z, vT):
    evaluation = kde_gaia_epanechnikov(np.array([[R, phi, z, vR, vT, vz]]))
    return evaluation

def integrate_over_vT(kde_gaia_epanechnikov, vT): #bounds of vR = [-100, 100], bounds of vz = [-100, 100]
    print ("Evaluating at vT =", vT)
    return dblquad(kde_gaia_cyl_vT, -100, 100, -100, 100, args=([8.3, 3.14, 0., vT]), epsabs=0.01)  # args=(R, phi, z, vR)

vT_output = np.ones([100])

print("With R, phi, z = 8.3, 3.14, 0., integrating over vR from [-100, 100] and vz from [-100, 100].")
print("KDE Evaluations along vT from [-400, 200] over 100 subintervals.")
print()

counter = 0
start_overall = time.time()

for v in np.nditer(v_input):
    counter += 1
    print("Evaluation:", counter)
    start = time.time()
    vT_output[counter - 1], error = integrate_over_vT(kde_gaia_epanechnikov, v)
    print("Value:", vT_output[counter - 1])
    end = time.time()
    print("Time to integrate: "'{}'"s".format(round(end - start, 2)))
    print("Time elapsed: "'{}'" min".format(round((end - start_overall)/60, 2)))
    print()
    
# -----------------
# Integrate over vz
# -----------------

def kde_gaia_cyl_vz(vR, vT, R, phi, z, vz):
    evaluation = kde_gaia_epanechnikov(np.array([[R, phi, z, vR, vT, vz]]))
    return evaluation

def integrate_over_vz(kde_gaia_epanechnikov, vz): #bounds of vR = [-100, 100], bounds of vT = [-300, 0]
    print ("Evaluating at vz =", vz)
    return dblquad(kde_gaia_cyl_vz, -100, 100, -300, 0, args=([8.3, 3.14, 0., vz]), epsabs=0.001)  # args=(R, phi, z, vR)

vz_output = np.ones([100])

print("With R, phi, z = 8.3, 3.14, 0., integrating over vR from [-100, 100] and vT from [0, 300].")
print("KDE Evaluations along vz from [-400, 200] over 100 subintervals.")
print()

counter = 0
start_overall = time.time()

for v in np.nditer(v_input):
    counter += 1
    print("Evaluation:", counter)
    start = time.time()
    vz_output[counter - 1], error = integrate_over_vz(kde_gaia_epanechnikov, v)
    print("Value:", vz_output[counter - 1])
    end = time.time()
    print("Time to integrate: "'{}'"s".format(round(end - start, 2)))
    print("Time elapsed: "'{}'" min".format(round((end - start_overall)/60, 2)))
    print()

In [ ]:
gaia_data2 = search_phase_space(0., 0., 0., 0., 0., 0., 0.05, 0.)
R, phi, z = rect_to_cyl(gaia_data2[:,0], gaia_data2[:,1], gaia_data2[:,2])
vR, vT, vz = rect_to_cyl_vec(gaia_data2[:,3], gaia_data2[:,4], gaia_data2[:,5], gaia_data2[:,0], gaia_data2[:,1], gaia_data2[:,2])
gaia_data2 = np.stack((R, phi, z, vR, vT, vz), axis=1)

In [ ]:
#initialize mean values for (R, vR, vT, z, vz)
mR,mz = 8.3/8. ,0./8. #convert to nat. units

vRs= np.linspace(-1,1,100) #odd endpoints because galpy uses natural units
vTs= np.linspace(0,2,100)
vzs= np.linspace(-1,1,100)

#Calculating the probability of vR without regard for vT or vz 
#is called marginalizing over the remaining probabilities.
#This is the opposite of conditional probability. 

pvR= np.array([qdfS.pvR(vR,mR,mz) for vR in vRs]) #uses Gauss-Legendre integration
pvT= np.array([qdfS.pvT(vT,mR,mz) for vT in vTs])
pvz= np.array([qdfS.pvz(vz,mR,mz) for vz in vzs])

#plot!
fig, ax = plt.subplots(1, 3, sharex='none', sharey='none', figsize=(20,5))

ax[0].plot(vRs*220, pvR/np.sum(pvR)/(vRs[1]-vRs[0])/220) #convert to physical units and normalize area to 1 
ax[0].set_title('vR', fontsize=20)
ax[1].plot(vTs*220, pvT/np.sum(pvT)/(vTs[1]-vTs[0])/220)
ax[1].set_title('vT', fontsize=20)
ax[2].plot(vRs*220, pvz/np.sum(pvR)/(vzs[1]-vzs[0])/220)
ax[2].set_title('vz', fontsize=20)


#above graphs overlayed
vs= np.linspace(-1,2,100)

pvR= np.array([qdfS.pvR(v,mR,mz) for v in vs])
pvT= np.array([qdfS.pvT(v,mR,mz) for v in vs])
pvz= np.array([qdfS.pvz(v,mR,mz) for v in vs])

fig, ax = plt.subplots(figsize=(10, 5))

ax.plot(vs*220, pvR/np.sum(pvR)/(vs[1]-vs[0])/220, label='vR') #convert to physical units and normalize area to 1 
ax.plot(vs*220, pvT/np.sum(pvT)/(vs[1]-vs[0])/220, label='vT')
ax.plot(vs*220, pvz/np.sum(pvz)/(vs[1]-vs[0])/220, label='vz')
ax.hist(vR, bins=100, normed=True, label='vR_histogram', alpha=0.5)
ax.hist(vT, bins=100, normed=True, label='vT_histogram', alpha=0.5)
ax.hist(vz, bins=100, normed=True, label='vz_histogram', alpha=0.5)
legend = ax.legend(loc='upper left', shadow=True, fontsize='x-large')
legend.get_frame().set_facecolor('#F0F0F0')
plt.show()

In [ ]:
# overplotting qdf and kde integrated
fig, ax = plt.subplots(figsize=(20, 10))

ax.plot(v_input, vR_output/np.sum(vR_output)/(v_input[1]-v_input[0]), label='vR_kde_integrated') #normalized area to 1
ax.plot(v_input, vT_output/np.sum(vT_output)/(v_input[1]-v_input[0]), label='vT_kde_integrated')
ax.plot(v_input, vz_output/np.sum(vz_output)/(v_input[1]-v_input[0]), label='vz_kde_integrated')
ax.plot(vs*220, pvR/numpy.sum(pvR)/(vs[1]-vs[0])/220, label='vR_qdf_integrated')
ax.plot(vs*220, pvT/numpy.sum(pvT)/(vs[1]-vs[0])/220, label='vT_qdf_integrated')
ax.plot(vs*220, pvz/numpy.sum(pvz)/(vs[1]-vs[0])/220, label='vz_qdf_integrated')
ax.hist(vR, bins=100, normed=True, label='vR_histogram', alpha=0.5)
ax.hist(vT, bins=100, normed=True, label='vT_histogram', alpha=0.5)
ax.hist(vz, bins=100, normed=True, label='vz_histogram', alpha=0.5)
legend = ax.legend(loc='upper left', shadow=True, fontsize='x-large')
legend.get_frame().set_facecolor('#F0F0F0')
plt.show()